# Web Scraping Mercado Livre


In [18]:
# Importando as bibliotecas

from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import numpy as np

In [19]:
URL = 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779362-1&page=1'
response = requests.get(URL)
page_contents = response.text

In [20]:
#Creating a file and loading the page contents in it.
with open('mercadolivre.html','w', encoding = "utf-8") as f:
    f.write(page_contents)

In [21]:
from bs4 import BeautifulSoup
doc = BeautifulSoup(page_contents,'html.parser')

In [22]:
# Pegando todos os itens da primeira página
def get_itens_title(doc):
    title_tags = doc.find_all('p', class_='promotion-item__title')
    titles = []
    for tags in title_tags:
        titles.append(tags.text)
    return titles

In [23]:
# Coletando os preços dos itens
def get_price(doc):
    price_tags = doc.find_all('div', class_ = 'andes-money-amount-combo__main-container')
    price = []
    for tags in price_tags:
        price.append(tags.text.replace('Â',''))
    return price

In [24]:
def get_doc(url):
    response = requests.get(url)
    doc = BeautifulSoup(response.text,'html.parser')
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(response))
    return doc

In [25]:
def scrape_multiple_pages(n):
    URL = 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779362-1&page='
    titles,prices= [],[]
    
    for page in range(1,n+1):
        doc = get_doc(URL + str(page))
        titles.extend(get_itens_title(doc))
        prices.extend(get_price(doc))
       
       
        
        
    itens = {'TITLE':titles,
                'PRICE':prices
            }
    return pd.DataFrame(itens)

In [26]:
df_mercado_livre = scrape_multiple_pages(10)
df_mercado_livre["SCRAPY_DATETIME"] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
df_mercado_livre['PRICE'] = df_mercado_livre['PRICE'].str.extract(r'(\d+[\.,]?\d*)')


In [27]:
df_mercado_livre

,TITLE,PRICE,SCRAPY_DATETIME
0,"Smart TV Philco PTV32G70RCH LED HD 32"" 110V/220V",1076,2023-02-21 11:08:26
1,Samsung Galaxy A03 Core Dual SIM 32 GB bronze ...,679,2023-02-21 11:08:26
2,Kit Whey Protein 2kg + Bcaa + Creatina + Shaker,91,2023-02-21 11:08:26
3,"Cabo Flexível 2,5mm Rolo 100m Metros Fio Elétr...",95,2023-02-21 11:08:26
4,Pistola Pintura Pulverizadora Tinta Paint Com ...,186,2023-02-21 11:08:26
...,...,...,...
475,"Cortina Sala Quarto Corta Luz 100% Blackout 2,...",147,2023-02-21 11:08:26
476,Kit 10 Luminária Solar Parede Sensor Presença ...,204,2023-02-21 11:08:26
477,Suplemento em pó Essential Nutrition Cacao Wh...,298,2023-02-21 11:08:26
478,Kit 10 Cuecas Boxer Microfibra Lisa Polo Wear ...,99,2023-02-21 11:08:26


# Armazenando os dados no banco de dados

In [28]:
import sqlite3
from sqlalchemy import create_engine

In [29]:
mercado_livre_schema = """
CREATE TABLE mercado_livre (
TITLE              TEXT,
PRICE              INTEGER,
SCRAPY_DATETIME    TEXT
)

"""

In [30]:
conn = create_engine( 'sqlite:///mercado_livre.sqlite', echo=False )

In [31]:
# connect to dataset
conn = sqlite3.connect('mercado_livre.sqlite')

#cursor = conn.execute(mercado_livre_schema)
#conn.commit()


In [32]:
# insert data to table
df_mercado_livre.to_sql('mercado_livre', con=conn, if_exists='append', index=False )

480

In [33]:
query = """
SELECT * FROM mercado_livre
"""
df_mercado_livre = pd.read_sql_query( query, conn)
df_mercado_livre

,TITLE,PRICE,SCRAPY_DATETIME
0,"Smart TV Philco PTV32G70RCH LED HD 32"" 110V/220V",1076,2023-02-21 11:07:27
1,Samsung Galaxy A03 Core Dual SIM 32 GB bronze ...,679,2023-02-21 11:07:27
2,Kit Whey Protein 2kg + Bcaa + Creatina + Shaker,91,2023-02-21 11:07:27
3,"Cabo Flexível 2,5mm Rolo 100m Metros Fio Elétr...",95,2023-02-21 11:07:27
4,Pistola Pintura Pulverizadora Tinta Paint Com ...,186,2023-02-21 11:07:27
...,...,...,...
955,"Cortina Sala Quarto Corta Luz 100% Blackout 2,...",147,2023-02-21 11:08:26
956,Kit 10 Luminária Solar Parede Sensor Presença ...,204,2023-02-21 11:08:26
957,Suplemento em pó Essential Nutrition Cacao Wh...,298,2023-02-21 11:08:26
958,Kit 10 Cuecas Boxer Microfibra Lisa Polo Wear ...,99,2023-02-21 11:08:26


In [34]:
df_mercado_livre[df_mercado_livre["TITLE"]=="Kit Whey Protein 2kg + Bcaa + Creatina + Shaker"]

,TITLE,PRICE,SCRAPY_DATETIME
2,Kit Whey Protein 2kg + Bcaa + Creatina + Shaker,91,2023-02-21 11:07:27
482,Kit Whey Protein 2kg + Bcaa + Creatina + Shaker,91,2023-02-21 11:08:26
